In [1]:
%%capture
# Install the latest version of attk (if necessary)
#!pip install -U git+git://github.com/hipstas/audio-tagging-toolkit.git

In [2]:
import attk
import os
import numpy as np
import librosa
from IPython.display import display, Audio
import timeit
import random
from itertools import groupby
from operator import itemgetter
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
import subprocess
import unicodecsv
import urllib2
import csv


os.chdir('/sharedfolder/GitHub/sida/')

!mkdir /sharedfolder/GitHub/sida/___training_audio

os.chdir('/sharedfolder/GitHub/sida/___training_audio')

## Uncomment lines below to download audio files for training

#!wget http://www.stephenmclaughlin.net/HILT/audio_corpora/NPR_Fresh_Air_diarized.zip
#!unzip NPR_Fresh_Air_diarized.zip

mkdir: cannot create directory '/sharedfolder/GitHub/sida/___training_audio': File exists


In [3]:
## Loading label data CSV as a list of lists

## Random 1-second labels
#csv_url = "https://raw.githubusercontent.com/hipstas/aapb-labels/master/Terry_Gross/Terry_Gross_labels.csv"

## Labels done by hand in Sonic Visualiser
csv_url = 'https://raw.githubusercontent.com/hipstas/aapb-labels/master/Terry_Gross/Terry_Gross_corrected_SV.csv'

csv_string = urllib2.urlopen(csv_url)

train_table = []

csv_reader = unicodecsv.reader(csv_string)

for row in csv_reader:
        train_table.append(row)

train_table[:10]+['...']

URLError: <urlopen error [Errno -2] Name or service not known>

In [4]:
## Removing header row if present

if 'Media file basename' in train_table[0]:
    train_table = train_table[1:]

random.shuffle(train_table)

NameError: name 'train_table' is not defined

In [ ]:
%%capture

## Excerpting WAV clips corresponding to labels

training_audio_pathname = "NPR_Fresh_Air_diarized"
out_dir = '_classes_SV_' + training_audio_pathname


for row in train_table:
    try:
        basename , start, duration, class_name, labeled_by = row  ## Assigning values in row to variables
        filename = str(basename + '.mp3')
        start = float(start)
        end = float(start) + float(duration)
        out_pathname = str(os.path.join(out_dir, class_name.replace(' ','_')))
        try: 
            subprocess.call(['mkdir', '-p', out_pathname])
        except:
            pass
        attk.subclip(os.path.join(training_audio_pathname, filename), float(start), end, out_pathname) ## <- attk
    except Exception as e: 
        print(row)
        print(e)



In [ ]:
################################################

In [5]:

def extract_pairs(media_path,vowel_time_ranges):
    snd = AudioFileClip.AudioFileClip(media_path)
    file_duration = attk.duration(media_path)
    for pair in vowel_time_ranges:
        start, end = pair
        start = float(start)
        end = float(end)
        if end-start >= 0.1:  ## Ignore clips shorter than 0.1 second
            basename = media_path.split('/')[-1][:-4]
            out_filename = basename+'__'+str(round(start, 4))+'_'+str(round(end, 4))+'.wav'
            snd.subclip(start, end).write_audiofile(os.path.join('_vowel_clips',out_filename))


def batch_extract_vowels(media_dir):

    starting_location = os.getcwd()
    
    os.chdir(media_dir)

    bin_2048_to_sec_constant = 0.046439909297052155
    
    try: os.mkdir('_vowel_clips')
    except: pass

    filenames=[item for item in os.listdir('./') if item[-4:].lower() in ('.mp3','.wav','.mp4')]

    tic=timeit.default_timer()

    for filename in filenames[::-1]:
        try:
            vowel_bools = attk.get_vowel_segments(filename)

            vowel_bin_ranges = attk.labels_to_ranges(vowel_bools, label=True)

            vowel_time_ranges = [(s*bin_2048_to_sec_constant, e*bin_2048_to_sec_constant) for s, e in vowel_bin_ranges]
            
            extract_pairs(filename,vowel_time_ranges)

        except: print("***** ERROR: "+filename)

    print("Time elapsed: "+str(timeit.default_timer() - tic))

    os.chdir(starting_location)


In [ ]:
%%capture

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_SV_NPR_Fresh_Air_diarized')

#for class_dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
#    batch_extract_vowels(class_dir_name)

batch_extract_vowels('Terry_Gross')
#batch_extract_vowels('Background_Speaker')

In [ ]:
%%capture

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_NPR_Fresh_Air_diarized')

#for class_dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
#    batch_extract_vowels(class_dir_name)

batch_extract_vowels('Terry_Gross')
batch_extract_vowels('Background_Speaker')

In [ ]:
%%capture

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017')

#batch_extract_vowels('Female')
batch_extract_vowels('Male')


In [11]:
## Extracting features
tic=timeit.default_timer()

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_SV_NPR_Fresh_Air_diarized/')


dir_names = [item for item in os.listdir('./') if os.path.isdir(item)]

for dir_name in dir_names:
    
    os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_SV_NPR_Fresh_Air_diarized/' + dir_name + '/_vowel_clips')
    
    try: os.mkdir('../_vowel_mfccs_and_deltas')
    except: pass
    
    for filename in [item for item in os.listdir('./') if item.lower()[-4:] == '.wav']:
        csv_out_path = '../_vowel_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv'
        if not os.path.isfile(csv_out_path):
            try:
                mfccs = attk.get_mfccs_and_deltas(filename)
                with open(csv_out_path, 'w') as fo:
                    csv_writer = csv.writer(fo)
                    csv_writer.writerows(mfccs)  
            except:
                "ERROR on " + filename


print(timeit.default_timer() - tic)

tic=timeit.default_timer()

##


11.505573988


In [12]:
## Extracting features
tic=timeit.default_timer()

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_NPR_Fresh_Air_diarized/')

dir_names = [item for item in os.listdir('./') if os.path.isdir(item)]

for dir_name in dir_names:
    
    try:
        os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_NPR_Fresh_Air_diarized/' + dir_name + '/_vowel_clips')
    
        try: os.mkdir('../_vowel_mfccs_and_deltas')
        except: pass
    
        csv_out_path = '../_vowel_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv'
        if not os.path.isfile(csv_out_path):
            try:
                mfccs = attk.get_mfccs_and_deltas(filename)
                with open(csv_out_path, 'w') as fo:
                    csv_writer = csv.writer(fo)
                    csv_writer.writerows(mfccs)  
            except:
                "ERROR on " + filename
                
    except: pass

print(timeit.default_timer() - tic)

tic=timeit.default_timer()

##


0.989500045776


In [13]:
## Extracting features
tic=timeit.default_timer()

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017')

dir_names = [item for item in os.listdir('./') if os.path.isdir(item)]

for dir_name in dir_names:
    
    try:
        os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017/' + dir_name + '/_vowel_clips')
    
        try: os.mkdir('../_vowel_mfccs_and_deltas')
        except: pass
    
        for filename in [item for item in os.listdir('./') if item.lower()[-4:] == '.wav']:
            mfccs = attk.get_mfccs_and_deltas(filename)
            with open('../_vowel_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv', 'w') as fo:
                csv_writer = csv.writer(fo)
                csv_writer.writerows(mfccs)  
    except: pass

print(timeit.default_timer() - tic)

tic=timeit.default_timer()

##


217.990756035


In [16]:

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_NPR_Fresh_Air_diarized/Terry_Gross/_vowel_mfccs_and_deltas')

gross_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            gross_features.append([float(item) for item in row])

print(len(gross_features))


os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_SV_NPR_Fresh_Air_diarized/Terry_Gross/_vowel_mfccs_and_deltas')

gross_sv_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            gross_sv_features.append([float(item) for item in row])

print(len(gross_sv_features))

3824
40301


In [17]:
import csv

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_NPR_Fresh_Air_diarized/Background_Speaker/_vowel_mfccs_and_deltas')

fa_ubm_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            fa_ubm_features.append([float(item) for item in row])

print(len(fa_ubm_features))



13314


In [18]:
import csv

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017/Male/_vowel_mfccs_and_deltas')

m_ubm_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            m_ubm_features.append([float(item) for item in row])

print(len(m_ubm_features))


os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017/Female/_vowel_mfccs_and_deltas')

f_ubm_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            f_ubm_features.append([float(item) for item in row])

print(len(f_ubm_features))

11531
4382


In [47]:
os.chdir('/sharedfolder/GitHub/sida/___training_audio/')

In [57]:
## Combining feature sets

terry_gross_mfccs = gross_features #+ gross_sv_features
ubm_mfccs = fa_ubm_features + m_ubm_features + f_ubm_features

#terry_gross_mfccs = terry_gross_mfccs[::-1]
#ubm_mfccs = ubm_mfccs[::-1]

#random.shuffle(terry_gross_mfccs)
#random.shuffle(ubm_mfccs)

print(len(terry_gross_mfccs))
print(len(ubm_mfccs))

3824
29227


In [58]:
## Decision tree

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier

tic=timeit.default_timer()

X = terry_gross_mfccs[:-len(terry_gross_mfccs)/10] + ubm_mfccs[:-len(ubm_mfccs)/10]
y = [0]*len(terry_gross_mfccs[:-len(terry_gross_mfccs)/10]) + [1]*len(ubm_mfccs[:-len(ubm_mfccs)/10])

X_test = terry_gross_mfccs[-len(terry_gross_mfccs)/10:] + ubm_mfccs[-len(ubm_mfccs)/10:]
y_test = [0]*len(terry_gross_mfccs[-len(terry_gross_mfccs)/10:]) + [1]*len(ubm_mfccs[-len(ubm_mfccs)/10:])

classifier = MLPClassifier().fit(X, y)

## Saving trained model
joblib.dump(classifier,'gross_vowels_et_2048.pkl')
classifier=joblib.load('gross_vowels_et_2048.pkl')

print(timeit.default_timer() - tic)

8.3721768856


In [59]:
classifier.score(X_test,y_test)

0.82637628554143983

In [72]:
## Combining feature sets

speaker_1_mfccs = gross_features + gross_sv_features
bg_mfccs = fa_ubm_features + m_ubm_features + f_ubm_features

#speaker_1_mfccs = terry_gross_mfccs[::-1]
#bg_mfccs = ubm_mfccs[::-1]

#random.shuffle(speaker_1_mfccs)
#random.shuffle(bg_mfccs)

print(len(terry_gross_mfccs))
print(len(ubm_mfccs))

3824
29227


In [73]:
### KERAS

import pandas as pd

speaker_1_mfccs_df = pd.DataFrame(speaker_1_mfccs[:-500])

speaker_1_mfccs_df['Class Label'] = 1.0   ## Terry Gross


bg_mfccs_df = pd.DataFrame(bg_mfccs[:-500])

bg_mfccs_df['Class Label'] = 0.0    ## Background speaker


training_data_df = pd.concat([speaker_1_mfccs_df, bg_mfccs_df])

training_data_df.head()



,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,Class Label
0,145.528022,52.556185,-19.836026,-4.779479,-1.471527,18.713974,-8.128435,-10.357780,-3.947624,1.606438,...,0.288286,-0.592404,-1.130820,0.004470,0.169714,0.092444,-0.516519,0.004081,-0.419400,1.0
1,143.162753,-22.091946,11.111120,-5.802959,-5.327405,4.877378,-13.537851,-2.446800,-15.543847,16.445061,...,0.640322,0.077595,-0.315508,0.754390,-0.619043,-0.025048,-1.225697,0.418339,0.236048,1.0
2,-0.469516,40.253117,75.463766,2.474142,16.307388,6.242862,-18.638111,-12.341957,2.357186,-12.481317,...,0.623618,0.571974,-0.249141,-0.044193,0.020489,0.339805,-0.032752,-0.429253,0.275497,1.0
3,179.407453,-12.910994,5.877851,-53.849837,-3.449369,14.889508,10.798665,-36.533244,-7.673434,-5.605504,...,0.554404,-0.165666,0.107095,0.232014,-0.385006,0.280536,-0.905222,0.602623,0.441850,1.0
4,158.184824,68.084102,31.297796,36.776827,28.833738,-10.733498,11.227472,-6.689189,0.528000,16.571019,...,-1.621358,-1.651220,0.495750,-0.225871,0.120457,-0.323987,-0.776101,0.168192,-0.311773,1.0


In [74]:
speaker_1_mfccs_df_test = pd.DataFrame(speaker_1_mfccs[-500:])
speaker_1_mfccs_df_test['Class Label'] = 1.0   ## Terry Gross

bg_mfccs_df_test = pd.DataFrame(bg_mfccs[-500:])
bg_mfccs_df_test['Class Label'] = 0.0    ## Background speaker

test_data_df = pd.concat([speaker_1_mfccs_df_test, bg_mfccs_df_test])

In [75]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well.
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
scaled_training = scaler.fit_transform(training_data_df)
scaled_testing = scaler.transform(test_data_df)

# Print out the adjustment that the scaler applied to the total_earnings column of data
#######print("Note: total_earnings values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[8], scaler.min_[8]))

# Create new pandas DataFrame objects from the scaled data
scaled_training_df = pd.DataFrame(scaled_training, columns=training_data_df.columns.values)
scaled_testing_df = pd.DataFrame(scaled_testing, columns=test_data_df.columns.values)

scaled_testing_df.head()

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,Class Label
0,0.542788,0.387842,0.588833,0.542590,0.455971,0.483816,0.634891,0.476033,0.479879,0.499572,...,0.475239,0.444128,0.653842,0.510107,0.412542,0.415514,0.545748,0.527017,0.622477,1.0
1,0.620554,0.427259,0.424873,0.575730,0.487262,0.569304,0.514076,0.518479,0.565739,0.627275,...,0.448298,0.498846,0.399751,0.400400,0.367423,0.331029,0.326291,0.547375,0.461277,1.0
2,0.801378,0.412346,0.540283,0.515692,0.536305,0.439811,0.429129,0.363124,0.427275,0.480368,...,0.455441,0.494850,0.400423,0.384490,0.353950,0.322277,0.330851,0.526404,0.431842,1.0
3,0.606114,0.377198,0.657624,0.568292,0.524360,0.541956,0.714307,0.526312,0.489528,0.524479,...,0.486767,0.339382,0.343371,0.113690,0.433394,0.423659,0.412654,0.621854,0.537593,1.0
4,0.702619,0.259012,0.490942,0.535733,0.428951,0.327187,0.439968,0.450061,0.504347,0.476029,...,0.505922,0.300177,0.322718,0.093326,0.425267,0.402008,0.392605,0.599755,0.541202,1.0


In [76]:

# Save scaled data dataframes to new CSV files
scaled_training_df.to_csv("terry_gross_vs_ubm_train_scaled.csv", index=False)
scaled_testing_df.to_csv("terry_gross_vs_ubm_test_scaled.csv", index=False)

In [ ]:
tic=timeit.default_timer()

import pandas as pd
from keras.models import Sequential
from keras.layers import *

training_data_df = pd.read_csv("terry_gross_vs_ubm_train_scaled.csv")

X = training_data_df.drop('Class Label', axis=1).values
Y = training_data_df[['Class Label']].values

# Define the model
model = Sequential()
model.add(Dense(100, input_dim=38, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(42, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss="mean_squared_error", optimizer="adam")


print(timeit.default_timer() - tic)

1.33203291893


In [ ]:

# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2
)

# Load the separate test data set
test_data_df = pd.read_csv("terry_gross_vs_ubm_test_scaled.csv")

X_test = test_data_df.drop('Class Label', axis=1).values
Y_test = test_data_df[['Class Label']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: " + str(test_error_rate))

# Save the model to disk
model.save("trained_model.h5")
print("Model saved to disk.")




Epoch 1/50
6s - loss: 0.1844
Epoch 2/50
5s - loss: 0.1593
Epoch 3/50
5s - loss: 0.1512
Epoch 4/50
5s - loss: 0.1462
Epoch 5/50
5s - loss: 0.1426
Epoch 6/50
5s - loss: 0.1390
Epoch 7/50
5s - loss: 0.1366
Epoch 8/50
5s - loss: 0.1341
Epoch 9/50
5s - loss: 0.1323
Epoch 10/50
5s - loss: 0.1302
Epoch 11/50
5s - loss: 0.1276
Epoch 12/50
5s - loss: 0.1262
Epoch 13/50
6s - loss: 0.1249
Epoch 14/50
6s - loss: 0.1231
Epoch 15/50
6s - loss: 0.1220
Epoch 16/50
6s - loss: 0.1209
Epoch 17/50
6s - loss: 0.1197
Epoch 18/50
7s - loss: 0.1182
Epoch 19/50
7s - loss: 0.1172
Epoch 20/50
8s - loss: 0.1156
Epoch 21/50
6s - loss: 0.1151
Epoch 22/50
5s - loss: 0.1145
Epoch 23/50
5s - loss: 0.1130
Epoch 24/50
8s - loss: 0.1122
Epoch 25/50
6s - loss: 0.1106
Epoch 26/50
7s - loss: 0.1102
Epoch 27/50


In [ ]:
test_error_rate

In [ ]:

def classify_clip(clip_pathname):
    mfccs=get_mfccs_and_deltas(clip_pathname)
    results = random_forest.predict(mfccs)  ## Predicting new observation
    vowel_results=[]
    vowel_bools = get_vowel_segments(clip_pathname)[::2]

    for i in range(len(results)):
        if vowel_bools[i]==True:
            vowel_results.append(results[i])

    return np.mean(vowel_results) ## Vowels only


In [ ]:
## Loading pre-trained model

#from sklearn.ensemble import RandomForestClassifier

#random_forest=joblib.load('pesca_vowels_random_forest_2048.pkl')

In [ ]:

def classify_clip(clip_pathname):
    mfccs=get_mfccs_and_deltas(clip_pathname)
    results = classifier.predict(mfccs)  ## Predicting new observation
    vowel_results=[]
    vowel_bools = get_vowel_segments(clip_pathname)[::2]

    for i in range(len(results)):
        if vowel_bools[i]==True:
            vowel_results.append(results[i])

    return np.mean(vowel_results) ## Vowels only


In [ ]:
## Classifying short clips


tic=timeit.default_timer()

filename = random.choice(os.listdir('unseen/'))
test_pathname = 'unseen/'+filename
test_mfccs=attk.get_mfccs_and_deltas(test_pathname)

print(test_pathname)

results = classifier.predict(test_mfccs)  ## Predicting new observation

print(results)


vowel_results=[]

vowel_bools = get_vowel_segments(test_pathname)[::2]

for i in range(len(results)):
    if vowel_bools[i]==True:
        vowel_results.append(results[i])

display(Audio(test_pathname))


print("All: "+str(np.mean(results)))
print("Vowels only: "+str(np.mean(vowel_results)))

#print("Time elapsed: "+str(timeit.default_timer() - tic))

In [ ]:
len(get_vowel_segments(test_pathname))

In [ ]:
!du /tmp/temp_clip.wav

In [ ]:


## Classifying a long audio file


tic=timeit.default_timer()


media_path = "/sharedfolder/fa_eval/FA_Author_Tom_Perrotta.mp3"


snd = AudioFileClip.AudioFileClip(media_path)

classifications=[]

for i in range(int(attk.duration(media_path))):
    try:
        snd.subclip(i,i+1).write_audiofile('/tmp/temp_clip.wav')
        classifications.append(classify_clip('/tmp/temp_clip.wav'))
    except: print('missed one')

classifications

In [ ]:
# Writing classification output to CSV

counter=0

class_0_secs=[]
class_1_secs=[]

i=0

for classification in attk.smooth(np.array(classifications)):
    if classification < 0.34:
        class_0_secs.append(i)
    if classification > 0.38:
        class_1_secs.append(i)
    i+=1


counter=0

csv_path=media_path[:-4]+'_extratrees2048_labels.csv'

with open(csv_path,'w') as fo:
    for pair in seconds_list_to_ranges(class_0_secs):
        fo.write(str(float(pair[0]))+','+str(float(pair[1]))+',Terry Gross\n')
    for pair in seconds_list_to_ranges(class_1_secs):
        fo.write(str(float(pair[0]))+','+str(float(pair[1]))+',Background\n')


print(timeit.default_timer() - tic)